In [1]:
from __future__ import division
# import pyspark as ps    # import the spark suite
# import warnings         # display warning if spark context already exists
# import os
import pandas as pd
import numpy as np

from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, FloatType
from pyspark.sql.functions import mean as sql_mean

import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import colorsys
import random

import matplotlib.pyplot as plt
% matplotlib inline

## Now that our database is saved as a parquet, we can query it directly

In [5]:
# Use formatting to set table name

table = "parquet.`data/excerpt_df.parquet`"

spark.sql('''
    SELECT author, tokenized, w2v 
    FROM {}
    LIMIT 10
    '''.format(table)).show()

+--------------+--------------------+--------------------+
|        author|           tokenized|                 w2v|
+--------------+--------------------+--------------------+
|CharlesDickens|[a, christmas, ca...|[-0.1702252033501...|
|CharlesDickens|[mind, i, dont, m...|[-0.2174013902028...|
|CharlesDickens|[scrooge, never, ...|[-0.1526826002821...|
|CharlesDickens|[nobody, ever, st...|[-0.1897002885570...|
|CharlesDickens|[the, door, of, s...|[-0.1829826451034...|
|CharlesDickens|[i, do, said, scr...|[-0.2629524400289...|
|CharlesDickens|[uncle, pleaded, ...|[-0.2502119612455...|
|CharlesDickens|[the, clerk, in, ...|[-0.2832240152638...|
|CharlesDickens|[i, am, sorry, wi...|[-0.1989532466069...|
|CharlesDickens|[mr, marley, has,...|[-0.2183633688141...|
+--------------+--------------------+--------------------+



In [6]:
# thankyou nick for the plotting code

def make_plots(DFs, col, bins, title, colors, x_lab, authors, x_lim=None):
    plt.subplots(4,4,sharex=True, sharey=True, figsize=(8,8))
    plt.tight_layout()
    plt.subplots_adjust(top=0.94)
    plt.suptitle(title, fontsize=18)
    for i, DF in enumerate(DFs):
        make_hist(DF, col, bins, colors[i], x_lim, i, x_lab, authors[i])
    
    
def make_hist(DF, col, bins, color, x_lim, i, x_lab, author):
    # Create pandas DF of bins and counts
    temp_df = DF.select(col).toPandas()
    
    plt.subplot(2,2,i+1)
    weights = temp_df.Frequency/temp_df.Frequency.sum()
    plt.hist(temp_df[col], bins, normed=1, color=color, 
             alpha=0.2, label=author)
    # Find mean sentence length
    x_val = DF.select(sql_mean(col)).select('avg('+col+')').head()[0]
    plt.axvline(x=x_val, color=color, label="Mean")
    plt.xlabel(x_lab)
    plt.ylabel("Frequency")
    plt.xlim(x_lim)
    plt.legend();

In [23]:
print 'hello {}, are you a {}?'.format('evan', 'duck')

author = "'MarkTwain'"

spark.sql('''
        SELECT author, excerpt, char_count, avg_wordlen, word_count
        FROM {} 
        WHERE author = {} AND sent_length <=100
        '''.format(table, author)).show()

# spark.sql('''
#         SELECT char_count, avg_wordlen, word_count, sent_length 
#         FROM parquet.`data/excerpt_df.parquet` 
#         WHERE author = 'MarkTwain' AND sent_length <=100
#         ''').show()

hello evan, are you a duck?
+---------+--------------------+----------+-----------+----------+
|   author|             excerpt|char_count|avg_wordlen|word_count|
+---------+--------------------+----------+-----------+----------+
|MarkTwain|A CONNECTICUT YAN...|    1275.0|   4.425532|     235.0|
|MarkTwain|“You know about t...|    1529.0|   4.687732|     269.0|
|MarkTwain|HOW SIR LAUNCELOT...|    3034.0|   4.188034|     585.0|
|MarkTwain|And then they all...|    1574.0|  3.9684544|     317.0|
|MarkTwain|As I laid the boo...|    1211.0|   4.454955|     222.0|
|MarkTwain|Well, a man like ...|    1224.0|   4.212766|     235.0|
|MarkTwain|“Fair sir, will y...|    1499.0|  3.8076923|     312.0|
|MarkTwain|“Bridgeport?” sai...|    1197.0|   4.396396|     222.0|
|MarkTwain| || THE TALE OF T...|    1999.0|   4.203125|     384.0|
|MarkTwain|As we approached ...|    2457.0|   4.729604|     429.0|
|MarkTwain| || CHAPTER II ||...|    1646.0|    4.14375|     320.0|
|MarkTwain|“Go ‘long,” I sai...|  

In [7]:
def metadata_df(author):
    return spark.sql('''
        SELECT character_count, avg_wordlen, word_count, sent_length 
        FROM {} 
        WHERE author = {} AND sent_length <=100'''.format(table, author)).persist()



twaindf = metadata_df('MarkTwain')
muirdf = metadata_df('JohnMuir')
austendf = metadata_df('JaneAusten')
dickensdf = metadata_df('CharlesDickens')


AnalysisException: u"cannot resolve '`MarkTwain`' given input columns: [sent_count, tfidf, avg_wordlen, word_count, count_vectorized, words_only, excerpt, title, author, sent_length, char_count, tokenized, w2v]; line 4 pos 23"